In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
from tqdm import tqdm
import pickle
import os
from tensorflow import keras
from tensorflow.keras import backend as K
import tensorflow as tf
import socket
from scipy.stats import norm

os.environ["CUDA_VISIBLE_DEVICES"]="2,3" #don't use GPU because it's in use. Need to change this every time


if 'biochem1' in socket.gethostname():
    dataPBase = '/avicenna/vramani/analyses/pacbio/'
    figPBase = '/avicenna/cmcnally/pbanalysis/'
if 'titan' in socket.gethostname():
    dataPBase = '/data/users/goodarzilab/colin/results/pacbio/'
if 'wynton' in socket.gethostname():
    dataPBase = '/wynton/group/goodarzilab/ramanilab/results/pacbio/'
if 'rumi' in socket.gethostname():
    raise Exception('no pacbio results folder on rumi')
    
sampleRef = pd.read_csv(dataPBase + 'sampleRef_K562_mESC.csv', sep=',', index_col=0)

## Generate input formatted for tensorflow
First, split up the full files into smaller blocks if they are too large. Then run the script formatNNinput.py on each block.

In [ ]:
# split up full files for SAMv2
import os

usesamples = [39,43,47]

zmwPerBlock = 50000

for samp in usesamples:
    with open(os.path.join(dataPBase, sampleRef['cell'][samp],'processed','full',
                           sampleRef['cell'][samp] + '_' + sampleRef['sampleName'][samp] + '_full.pickle'), 'rb') as fin:
        ipdfull = pickle.load(fin, encoding="latin1")
        
    zmws = sorted(list(ipdfull.keys()))
    
    nBlock = int(np.ceil(len(zmws) / zmwPerBlock))
    
    for block in tqdm(np.arange(1,nBlock+1), position=0, desc=sampleRef['sampleName'][samp]):
        ipdsub = {}
        bstart = (block-1) * zmwPerBlock
        bend = min(block * zmwPerBlock, len(zmws))
        for zmw in zmws[bstart:bend]:
            ipdsub[zmw] = ipdfull[zmw]
        outfile = os.path.join(dataPBase, sampleRef['cell'][samp],'processed','full',
                               sampleRef['cell'][samp] + '_' + sampleRef['sampleName'][samp] + '_full_block{:03d}.pickle'.format(block))
        with open(outfile, 'wb') as fout:
            pickle.dump(ipdsub, fout, pickle.HIGHEST_PROTOCOL)
            
# check how many block files there are for each sample
import glob

usesamples = [39, 43, 47]
nblock = {}
for samp in usesamples:
    gl = glob.glob(dataPBase + '{0}/processed/full/{0}_{1}_full_block*.pickle'.format(sampleRef['cell'][samp],
                                                                                      sampleRef['sampleName'][samp]))
    nblock[samp] = len(gl)
    
sampL = []
blockL = []

for samp in usesamples:
    for block in np.arange(1,nblock[samp]+1):
        sampL.append(samp)
        blockL.append(block)
        
if not os.path.exists('/data/users/goodarzilab/colin/quickScripts'):
    os.makedirs('/data/users/goodarzilab/colin/quickScripts')
    
    
nScripts = 4
perS = int(np.ceil(len(blockL)/nScripts))

ix = 0

for scriptN in range(nScripts):
    with open('/data/users/goodarzilab/colin/quickScripts/makeNNinput_p{0}.sh'.format(scriptN), 'w') as fout:
        fout.write('#!/usr/bin/zsh\n\n')
        fout.write('source /data/users/goodarzilab/colin/.zshrc\n\n')
        fout.write('conda activate NN2\n\n')
        for ix in np.arange(scriptN*perS, min((scriptN+1)*perS, len(sampL))):
            fout.write('python ~/code/scripts/formatNNinput.py {0} {1}\n'.format(sampL[ix], blockL[ix]))

## Train neural network components of the model
First, a network to predict the IPD based on sequence context and non-adenine IPD percentiles in the molecule

A cutoff of 0.42 on the residuals of the observed IPD minus these predictions defines methylation

Second, a network to predict residuals in fully methylated controls, used to ignore sequence contexts that are rarely methylated

Third, a model is trained to predict the probability of an adenine in a sequence context being predicted as methylated in fully methylated control DNA. Used as the HMM emmission probability for the accessible state

Fourth, a model is trained to predict the probability of an adenine in a sequence context being predicted as methylated in unmethylated control DNA. Used as the HMM emmission probability for the inaccessible state

In [ ]:
# how many adenines are in each sample?
# for choosing how many adenines to use to train neural networks

for samp in range(30):
    try:
        with np.load(os.path.join(dataPBase, sampleRef['cell'][samp],'processed','forNN',
                                  sampleRef['cell'][samp] + '_' + sampleRef['sampleName'][samp] + '_forNN.npz')) as data:
            na = data['sampleArr'].shape[0]
        print('%02d: %s: %d' % (samp, sampleRef['sampleName'][samp], na))
    except:
        pass

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

def trainIPDmodel():
    ### Load in unmethylated controls to train IPD model
    usesamples = [6,7,10,11,12,13,16,17] #np.arange(8)
    aPerSample = {samp:5000000 for samp in usesamples} #{2:5000000, 3:3000000, 6:5000000, 7:3000000} #{samp:2000000 for samp in usesamples}
    ipdDat = []
    sampDat = []
    contextDat = []
    percsDat = []
    
    for samp in tqdm(usesamples, position=0):
        with np.load(os.path.join(dataPBase,
                                  sampleRef['cell'][samp],'processed','forNN',
                                  sampleRef['cell'][samp] + '_' + sampleRef['sampleName'][samp] + '_forNN.npz')) as data:
            ipdDat.append(data['ipdArr'][0:aPerSample[samp],:])
            sampDat.append(data['sampleArr'][0:aPerSample[samp],:])
            contextDat.append(data['contextmat'][0:aPerSample[samp],:])
            percsDat.append(data['percsmat'][0:aPerSample[samp],:][:,9:])

    ipdDat = np.vstack(ipdDat)
    sampDat = np.vstack(sampDat)
    contextDat = np.vstack(contextDat)
    percsDat = np.vstack(percsDat)

    ### Create model to predict IPD
    trainSet = np.nonzero(np.isin(sampDat, [6,10,11,12,16,17]))[0]
    validSet = np.nonzero(np.isin(sampDat, [7,13]))[0]

    context_inputs = keras.layers.Input(shape=contextDat.shape[1:])
    perc_input = keras.layers.Input(shape=percsDat.shape[1:])
    conc = keras.layers.concatenate([context_inputs, perc_input])
    lay1 = keras.layers.Dense(200, activation="relu", kernel_initializer='he_uniform')(conc)
    lay2 = keras.layers.Dense(200, activation="relu", kernel_initializer='he_uniform')(lay1)
    lay3 = keras.layers.Dense(200, activation="relu", kernel_initializer='he_uniform')(lay2)
    lay4 = keras.layers.Dense(200, activation="relu", kernel_initializer='he_uniform')(lay3)
    outputs = keras.layers.Dense(1)(lay4)
    ipdModel = keras.models.Model(inputs=[context_inputs, perc_input],
                               outputs=[outputs])

    optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
    ipdModel.compile(loss="mean_squared_error", optimizer=optimizer)

    ### Train the model on negative control rep 1s, with rep 2s as validation data
    early_stopping = EarlyStopping(patience = 2, restore_best_weights=True)

    history = ipdModel.fit([contextDat[trainSet,:], percsDat[trainSet,:]], ipdDat[trainSet,:],
                           epochs=20, batch_size=128, shuffle=True,
                           validation_data=([contextDat[validSet,:], percsDat[validSet,:]], ipdDat[validSet,:]),
                           callbacks=[early_stopping])
    
    return ipdModel

ipdModel = trainIPDmodel()

In [ ]:
# Train a model to predict the residuals of fully methylated DNA based on sequence
# Used to discriminate sequence contexts that are infrequently methylated
def trainPosResidModel(ipdModel):
    samp = 28
    with np.load(os.path.join(dataPBase,sampleRef['cell'][samp],'processed','forNN',
                              sampleRef['cell'][samp] + '_' + sampleRef['sampleName'][samp] + '_forNN.npz')) as data:
        ipdDat = data['ipdArr']
        #sampDat.append(data['sampleArr'][0:aPerSample[samp],:])
        contextDat = data['contextmat']
        percsDat = data['percsmat'][:,9:]

    predIPD = ipdModel.predict([contextDat, percsDat], batch_size=2048)
    resid = ipdDat - predIPD

    trainSet = np.arange(ipdDat.shape[0])
    np.random.shuffle(trainSet)
    print(trainSet[0:10])
    validSet = trainSet[30000000:40000000]
    trainSet = trainSet[0:20000000]
    
    context_inputs = keras.layers.Input(shape=contextDat.shape[1:])
    lay1 = keras.layers.Dense(100, activation="relu", kernel_initializer='he_uniform')(context_inputs)
    lay2 = keras.layers.Dense(100, activation="relu", kernel_initializer='he_uniform')(lay1)
    lay3 = keras.layers.Dense(100, activation="relu", kernel_initializer='he_uniform')(lay2)
    outputs = keras.layers.Dense(1)(lay3)
    posResidModel = keras.models.Model(inputs=[context_inputs],
                               outputs=[outputs])

    optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
    posResidModel.compile(loss="mean_squared_error", optimizer=optimizer)

    history = posResidModel.fit([contextDat[trainSet,:]], resid[trainSet,:],
                                epochs=2, batch_size=128, shuffle=True,
                                validation_data=([contextDat[validSet,:]], resid[validSet,:]))
    
    return posResidModel

posResidModel = trainPosResidModel(ipdModel)

In [4]:
samp = 4
ipdModel = keras.models.load_model(dataPBase + '%s/processed/NNmodels/ipdModel' % (sampleRef['cell'][samp]))
posResidModel = keras.models.load_model(dataPBase + '%s/processed/NNmodels/posResidualModel' % (sampleRef['cell'][samp]))

In [ ]:
# train model to predict the probability of a adenine being called as methylated in pos control
def trainPosProbModel(ipdModel, posResidModel):
    samp = 28
    with np.load(os.path.join(dataPBase,sampleRef['cell'][samp],'processed','forNN',
                              sampleRef['cell'][samp] + '_' + sampleRef['sampleName'][samp] + '_forNN.npz')) as data:
        ipdDat = data['ipdArr']
        #sampDat.append(data['sampleArr'][0:aPerSample[samp],:])
        contextDat = data['contextmat']
        percsDat = data['percsmat'][:,9:]

    predIPD = ipdModel.predict([contextDat, percsDat], batch_size=65536)
    resid = ipdDat - predIPD
    
    methPred = resid > 0.42
    
    resPred = posResidModel.predict([contextDat], batch_size=65536)
    
    userp = np.nonzero(resPred > 0.6)[0]
    print(userp.shape)
    np.random.shuffle(userp)
    trainSet = userp[0:int(len(userp) * .75)]
    validSet = userp[int(len(userp) * .8):int(len(userp) * .99)]
    print(trainSet.shape)
    print(validSet.shape)
    
    context_inputs = keras.layers.Input(shape=contextDat.shape[1:])
    lay1 = keras.layers.Dense(200, activation="relu", kernel_initializer='he_uniform')(context_inputs)
    lay2 = keras.layers.Dense(200, activation="relu", kernel_initializer='he_uniform')(lay1)
    lay3 = keras.layers.Dense(200, activation="relu", kernel_initializer='he_uniform')(lay2)
    outputs = keras.layers.Dense(1, activation="sigmoid")(lay3)
    posProbModel = keras.models.Model(inputs=[context_inputs],
                               outputs=[outputs])

    optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
    posProbModel.compile(loss="binary_crossentropy", optimizer=optimizer)

    for bsize in [256, 1024, 4096, 16384, 32768, 65536, 131072]:
        history = posProbModel.fit([contextDat[trainSet,:]], methPred[trainSet,:],
                                    epochs=1, batch_size=bsize, shuffle=True,
                                    validation_data=([contextDat[validSet,:]],
                                                     methPred[validSet,:]))
    
    return posProbModel
    
posProbModel = trainPosProbModel(ipdModel, posResidModel)

In [ ]:
# train model to predict the probability of a adenine being called as methylated in neg controls
def trainNegProbModel(ipdModel, posResidModel):
    usesamples = [6,11,13,16] #np.arange(8)
    aPerSample = {samp:12500000 for samp in usesamples} #{2:5000000, 3:3000000, 6:5000000, 7:3000000} #{samp:2000000 for samp in usesamples}
    ipdDat = []
    contextDat = []
    percsDat = []
    
    for samp in tqdm(usesamples, position=0):
        with np.load(os.path.join(dataPBase,
                                  sampleRef['cell'][samp],'processed','forNN',
                                  sampleRef['cell'][samp] + '_' + sampleRef['sampleName'][samp] + '_forNN.npz')) as data:
            ipdDat.append(data['ipdArr'][0:aPerSample[samp],:])
            contextDat.append(data['contextmat'][0:aPerSample[samp],:])
            percsDat.append(data['percsmat'][0:aPerSample[samp],:][:,9:])

    ipdDat = np.vstack(ipdDat)
    contextDat = np.vstack(contextDat)
    percsDat = np.vstack(percsDat)
    print('Finished loading')
    
    predIPD = ipdModel.predict([contextDat, percsDat], batch_size=65536)
    resid = ipdDat - predIPD
    
    methPred = resid > 0.42
    
    resPred = posResidModel.predict([contextDat], batch_size=65536)
    
    userp = np.nonzero(resPred > 0.6)[0]
    print(userp.shape)
    
    trainSet = userp[0:int(len(userp) * .75)]
    validSet = userp[int(len(userp) * .8):int(len(userp) * .99)]
    print(trainSet.shape)
    print(validSet.shape)
    
    context_inputs = keras.layers.Input(shape=contextDat.shape[1:])
    lay1 = keras.layers.Dense(200, activation="relu", kernel_initializer='he_uniform')(context_inputs)
    lay2 = keras.layers.Dense(200, activation="relu", kernel_initializer='he_uniform')(lay1)
    lay3 = keras.layers.Dense(200, activation="relu", kernel_initializer='he_uniform')(lay2)
    outputs = keras.layers.Dense(1, activation="sigmoid")(lay3)
    negProbModel = keras.models.Model(inputs=[context_inputs],
                               outputs=[outputs])

    optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
    negProbModel.compile(loss="binary_crossentropy", optimizer=optimizer)

    for bsize in [256, 1024, 4096, 16384, 32768, 65536, 131072]:
        history = negProbModel.fit([contextDat[trainSet,:]], methPred[trainSet,:],
                                    epochs=1, batch_size=bsize, shuffle=True,
                                    validation_data=([contextDat[validSet,:]],
                                                     methPred[validSet,:]))
    
    return negProbModel
    
negProbModel = trainNegProbModel(ipdModel, posResidModel)

In [ ]:
samp = 4
if not os.path.exists(dataPBase + '%s/processed/NNmodels' % (sampleRef['cell'][samp])):
    os.makedirs(dataPBase + '%s/processed/NNmodels' % (sampleRef['cell'][samp]))
ipdModel.save(dataPBase + '%s/processed/NNmodels/ipdModel' % (sampleRef['cell'][samp]))
posResidModel.save(dataPBase + '%s/processed/NNmodels/posResidualModel' % (sampleRef['cell'][samp]))
posProbModel.save(dataPBase + '%s/processed/NNmodels/posProbModel' % (sampleRef['cell'][samp]))
negProbModel.save(dataPBase + '%s/processed/NNmodels/negProbModel' % (sampleRef['cell'][samp]))

## Write out the HMM input for each molecule
Uses the above trained models to predict methylation, get probabiltiy of methylation if the base was accessible or inaccessible, and filters out rarely methylated sequence contexts. Writes this input into a pickle file that will be loaded by runAccessibilityHMM.py

In [3]:
# Make data formatted for HMM application

from glob import glob

samp = 4
ipdModel = keras.models.load_model(dataPBase + '%s/processed/NNmodels/ipdModel' % (sampleRef['cell'][samp]))
posResidModel = keras.models.load_model(dataPBase + '%s/processed/NNmodels/posResidualModel' % (sampleRef['cell'][samp]))
posProbModel = keras.models.load_model(dataPBase + '%s/processed/NNmodels/posProbModel' % (sampleRef['cell'][samp]))
negProbModel = keras.models.load_model(dataPBase + '%s/processed/NNmodels/negProbModel' % (sampleRef['cell'][samp]))


def makeHMMinput(samp):
    with open(dataPBase + '%s/processed/full/%s_%s_full_zmwinfo.pickle' %
              (sampleRef['cell'][samp], sampleRef['cell'][samp], sampleRef['sampleName'][samp]), 'rb') as fin:
        zmwinfo = pickle.load(fin, encoding="latin1")

    parts = sorted(glob(os.path.join(dataPBase,sampleRef['cell'][samp],'processed','forNN',
                                     sampleRef['cell'][samp] + '_' + 
                                     sampleRef['sampleName'][samp] + '_forNN*.npz')))

    pieceN = 0

    if not os.path.exists(dataPBase + '%s/processed/forHMM' % (sampleRef['cell'][samp])):
            os.makedirs(dataPBase + '%s/processed/forHMM' % (sampleRef['cell'][samp]))

    for inputPart in tqdm(parts, position=0, desc=sampleRef['sampleName'][samp]):
        with np.load(inputPart) as data:
            ipdDat = data['ipdArr']
            contextDat = data['contextmat']
            percsDat = data['percsmat'][:,9:]
            zmwDat = data['zmwArr']
            zmwPosDat = data['zmwPos']

        predIPD = ipdModel.predict([contextDat, percsDat], batch_size=131072)
        resid = ipdDat - predIPD

        methPred = resid > 0.42

        resPred = posResidModel.predict([contextDat], batch_size=131072)

        posProb = posProbModel.predict([contextDat], batch_size=131072)
        negProb = negProbModel.predict([contextDat], batch_size=131072)

        usePred = np.nonzero(resPred > 0.6)[0] 

        hmmInput = {}
        ia = 0

        while ia < resPred.shape[0]:
            zmw = zmwDat[ia,0]
            istart = ia
            while ia < resPred.shape[0] and zmwDat[ia,0] == zmw:
                ia += 1
            iend = ia - 1

            zmwInd = np.arange(istart, iend + 1)
            goodInd = zmwInd[resPred[zmwInd,0] > 0.6]
            goodDat = pd.DataFrame({'pos':zmwPosDat[goodInd, 0], 'methPred':methPred[goodInd, 0], 'posProb':posProb[goodInd, 0],
                          'negProb':negProb[goodInd, 0]})
            goodDat.sort_values(axis=0, by='pos', inplace=True)
            goodDat.reset_index(inplace=True)

            if goodDat.shape[0] >= 2:
                hmmInput[zmw] = {'inDat':goodDat,
                                 'cclen':zmwinfo['cclen'][zmwinfo['zmw'] == zmw].iloc[0]}

            if len(list(hmmInput.keys())) >= 2000 or ia == resPred.shape[0]:
                with open(dataPBase + '{0}/processed/forHMM/{0}_{1}_forHMM_piece{2:05d}.pickle'.format(sampleRef['cell'][samp],
                                                                                                   sampleRef['sampleName'][samp],
                                                                                                   pieceN), 'wb') as fout:
                    pickle.dump(hmmInput, fout, protocol=4)
                pieceN += 1
                hmmInput = {}

In [ ]:
usesamples = [39,43,47]

for samp in usesamples:
    makeHMMinput(samp)

In [5]:
# write scripts of all pieces of all samples
from glob import glob

with open('/data/users/goodarzilab/colin/code/scripts/doHMM.sh', 'w') as fout:
    for samp in [39,43,47]:
        pieces = glob(os.path.join(dataPBase,sampleRef['cell'][samp],'processed','forHMM',
                              sampleRef['cell'][samp] + '_' + 
                              sampleRef['sampleName'][samp] + '_forHMM_piece*.pickle'))
        ipiece = 0
        for p in pieces:
            fout.write('qsub /wynton/home/goodarzi/cpmcnally/code/scripts/shellRunAccessibilityHMM.sh {0} {1}\n'.format(samp,
                                                                                                                        ipiece))
            ipiece += 1
